Run cells bellow to download and preprocess the **MovieLens20M** dataset.

Estimated time is around 45 minutes.

In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-20m.zip

--2023-04-20 10:04:28--  https://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  13.9MB/s    in 14s     

2023-04-20 10:04:43 (13.2 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [2]:
!unzip ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [2]:
import pandas as pd

All Items

In [4]:
m = pd.read_csv('ml-20m/movies.csv')
m = m[m.movieId.notnull()].reindex()
m['itemid']=m.movieId.apply(lambda x: str(int(x)))
m['product_name'] = m['title']
items = m[['itemid','product_name','genres']]
items.to_json('items.json')

Purchases and grouped purchases.

In [5]:
interactions = pd.read_csv('ml-20m/ratings.csv')
interactions = interactions[interactions.rating>=4.]
interactions = interactions.sort_values(['userId','timestamp'])
interactions['itemid'] = interactions['movieId'].apply(str)
interactions['userid'] = interactions['userId'].apply(str)
interactions['amount'] = 1
interactions['date'] = interactions['timestamp']
interactions[['itemid','userid','amount','date']]
interactions.to_json("purchases.json")
interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
iii.to_json('purchases_txt.json')

Keep only users with 5 or more interactions.

Purchases.

In [6]:
purchases=pd.read_json('purchases.json')
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
purchases_item_counts = purchases[['userid','itemid']]
purchases_user_counts = purchases[['userid','itemid']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
pu5=purchases_user_count[purchases_user_count.nr_of_purchases>=5]
purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
purchases_pu5.to_json('purchases_pu5.json')

Grouped purchases.

In [7]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userId','itemids']].drop_duplicates()
iii['userid']=iii['userId'].apply(str)
iii = iii[['userid','itemids']]
iii.to_json('purchases_txt_pu5.json')

/tmp/ipykernel_10892/126306790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))


Users.

In [8]:
iii['userid'].to_frame().to_json('users_pu5.json')

Items

In [9]:
items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")

Items sorted by number of interactions

In [10]:
purchases_item_count_pu5.to_json("items_sorted_pu5.json")

Users sorted by number of interactions

In [11]:
pu5.to_json("users_sorted_pu5.json")

Create train, val and test split.

In [3]:
users = pd.read_json('users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:10000]
val_users = shuffled_users.iloc[10000:20000]
train_users = shuffled_users.iloc[20000:]

#test_users.to_json("test_users.json")
print(test_users.head())
#val_users.to_json("val_users.json")
print(val_users.head())
#train_users.to_json("train_users.json") 
print(train_users.head())

len(train_users),len(val_users),len(test_users)

          userid
9483226    65567
18812883  130320
1125581     7677
5262991    36081
17249182  119275
          userid
15163263  104804
13123091   90664
11207540   77425
9316001    64428
13769119   95146
          userid
9141029    63177
17703308  122423
5668357    39001
1562218    10557
2448694    16563


(116677, 10000, 10000)

List of generated json files with preprocessed data.

In [13]:
!ls *.json

items.json	       purchases_pu5.json      train_users.json
items_pu5.json	       purchases_txt.json      users_pu5.json
items_sorted_pu5.json  purchases_txt_pu5.json  users_sorted_pu5.json
purchases.json	       test_users.json	       val_users.json


: 